## Fundamentals of Analog-to-Digital Conversion (ADC)
## Objectives

- **Quantization:** Observe how bit depth affects signal fidelity and the "staircase" effect.
- **SQNR:** Validate the $6.02N + 1.76$ dB rule.
- **Aliasing:** Understand why the HackRF needs a specific sampling rate to see 16 MHz.

---

## Process Gain Formulas

### 1. Bandwidth (BW) Process Gain

When we oversample a signal ($f_s > 2 \times BW_{signal}$) and then digitally filter it to a target bandwidth, we "throw away" the noise outside that bandwidth. This improves the Signal-to-Quantization-Noise Ratio (SQNR).

The formula for bandwidth process gain is:

$$
PG_{BW} = 10 \cdot \log_{10}\left( \frac{f_s / 2}{BW_{target}} \right)
$$

Where:
- $f_s / 2$: The Nyquist bandwidth (total noise bandwidth).
- $BW_{target}$: The bandwidth of interest.

**Effect:** Every time you double the sampling rate for a fixed signal bandwidth, you gain 3 dB of SQNR.

---

### 2. FFT Process Gain

When looking at a signal in the frequency domain, the FFT spreads the quantization noise across all FFT bins. The signal power remains in one (or a few) bins, making it appear "higher" above the noise floor.

The formula for FFT process gain is:

$$
PG_{FFT} = 10 \cdot \log_{10}\left( \frac{N_{FFT}}{2} \right)
$$

Where:
- $N_{FFT}$: The number of points in the FFT.

**Note:** This doesn't actually remove noise; it just increases our ability to detect small signals hidden in the noise floor by narrowing the "resolution bandwidth" of each bin.


## How to run the examples

To execute the Python script, click the 'Run This Cell' button above or press 'Shift + Enter'. The first time, make sure to execute the initialization part to load the required libraries.

## Init Packages

In [6]:
# Check if packages are installed and install if needed
import subprocess
import sys

def install_package(package):
    try:
        # Try to import the package
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        # If the package is not installed, install it
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# List of packages to check and install if needed
packages = ["numpy", "matplotlib", "ipywidgets", "scipy", "mplcursors"]

for package in packages:
    install_package(package)

# Importing after ensuring packages are installed
import numpy as np
import matplotlib.pyplot as plt
import mplcursors  # Import for click interactivity
from ipywidgets import widgets, interactive_output, VBox, HBox
from IPython.display import display, clear_output
from scipy.fft import fft, fftfreq, fftshift


numpy is already installed.
matplotlib is already installed.
ipywidgets is already installed.
scipy is already installed.
mplcursors is already installed.


## 1. Quantization, SNQR, and Processing Gain 

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive_output
import mplcursors


# Ensure you have run: !pip install ipympl mplcursors
%matplotlib widget

# --- 0. Memory Management ---
plt.close('all')  # <--- This kills the old figure and resets memory

f_signal = 997

# --- 2. Plotting Function ---
def plot_adc(bits, signal_amp, adc_range, target_bw, fs): # Added fs here
    # --- Parameters ---
    num_periods = 5
    # Calculate the exact stop time
    t_stop = num_periods / f_signal
    # Generate time and signal - Using 15*fs for a "pseudo-continuous" analog look
    t = np.arange(0, t_stop, 1/(100*fs))
 
    BW = target_bw
    
    # --- Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_signal * t)
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    indices_pre = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices_pre, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    error = signal - quantized
    
    # --- Metrics ---
    sqnr_calc = 10 * np.log10(np.var(signal) / (np.var(error) + 1e-15))
    sqnr_theory = 6.02 * bits + 1.76
    safe_bw = max(1, min(BW, fs/2))
    process_gain = 10 * np.log10(fs/2 / safe_bw)
    sqnr_pg = sqnr_theory + process_gain
    
    # --- Plotting ---
    plt.close('all')
    fig = plt.figure(figsize=(10, 8))
    
    # Subplot 1: Time Domain
    ax1 = plt.subplot(3, 1, 1)
    line_ana, = ax1.plot(t, signal, 'b--', alpha=0.4, label="Analog Input")
    line_dig = ax1.step(t, quantized, where='mid', color='red', label="Digital Output")[0]
    
    title_str = (f"LSB: {lsb:.4f} V | SQNR [TH]: {sqnr_theory:.2f} dB | "
                 f"SNQR [MEAS]: {sqnr_calc:.2f} dB\nPG: {process_gain:.2f} dB | Total: {sqnr_pg:.2f} dB")
    ax1.set_title(title_str, fontsize=10, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.set_ylabel("Magnitude (V)")
    ax1.grid(True)

    # --- ADD INTERACTIVE CURSOR ---
    cursor = mplcursors.cursor([line_ana, line_dig], hover=False)
    
    @cursor.connect("add")
    def _(sel):
        label = sel.artist.get_label()
        x, y = sel.target
        idx = (np.abs(t - x)).argmin()
        
        current_error = signal[idx] - quantized[idx]
        
        sel.annotation.set_text(
            f"Selected: {label}\n"
            f"Time: {x:.5f}s\n"
            f"Ana: {signal[idx]:.3f}V\n"
            f"Dig: {quantized[idx]:.3f}V\n"
            f"Error: {current_error:.4f}V"
        )
        sel.annotation.get_bbox_patch().set(fc="white", alpha=0.9, edgecolor='black')

    # Subplot 2: Quantization Error
    ax2 = plt.subplot(3, 1, 2, sharex=ax1)
    ax2.plot(t, error, color='black', lw=0.8)
    ax2.set_ylabel("Error (V)")
    ax2.set_ylim(-adc_range, adc_range)
    ax2.grid(True)

    # Subplot 3: Frequency Domain
    ax3 = plt.subplot(3, 1, 3)
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = (np.abs(np.fft.rfft(quantized))/(n/2) + 1e-10)
    ax3.plot(freqs, fft_mag, color='purple')
    ax3.axvspan(0, BW, color='green', alpha=0.1, label='Target BW')
    ax3.set_ylabel("Magnitude")
    ax3.set_xlabel("Frequency (Hz)")
    ax3.set_ylim(0, max(fft_mag)*1.2)
    ax3.grid(True)
    
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
bits_w = widgets.IntSlider(min=1, max=12, step=1, value=4, description='Bits', continuous_update=False)
amp_w = widgets.FloatSlider(min=0.1, max=5.0, step=0.1, value=2.0, description='Sig Amp', continuous_update=False)
range_w = widgets.FloatSlider(min=0.5, max=5.0, step=0.1, value=2.0, description='ADC Range', continuous_update=False)
fs_w = widgets.FloatSlider(value=f_signal*2, min=f_signal/10, max=f_signal*1000, step=1e6, description='fs (Hz)', continuous_update=False)
bw_w = widgets.FloatSlider(value=f_signal*2, min=f_signal/5, max=f_signal*50, step=1e5, description='BW (Hz)', continuous_update=False)

out = interactive_output(plot_adc, {
    'bits': bits_w, 
    'signal_amp': amp_w, 
    'adc_range': range_w, 
    'target_bw': bw_w,
    'fs': fs_w # Added fs mapping
})

ui = VBox([
    HBox([bits_w, amp_w, range_w]),
    HBox([fs_w, bw_w])
])

display(ui, out)

Output()

## 2. Aliasing & Nyquist

In [23]:
def plot_aliasing_colored(f_signal, fs):
    fn = fs / 2
    duration = 0.01
    t_analog = np.linspace(0, duration, 2000)
    t_samples = np.arange(0, duration, 1/fs)
    
    # Signal Generation
    sig_analog = 2.5 * np.sin(2 * np.pi * f_signal * t_analog)
    sig_samples = 2.5 * np.sin(2 * np.pi * f_signal * t_samples)
    
    # FFT Calculation
    N_fft = 2048
    yf = fftshift(fft(sig_samples, N_fft))
    xf = fftshift(fftfreq(N_fft, 1/fs))
    mag = 2.0/len(sig_samples) * np.abs(yf)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8))
    plt.subplots_adjust(hspace=0.4)

    # --- Time Domain ---
    ax1.plot(t_analog, sig_analog, label="Analog (RF)", alpha=0.3, color='gray')
    ax1.stem(t_samples, sig_samples, 'r', label="Samples", basefmt=" ")
    ax1.plot(t_samples, sig_samples, 'r--', alpha=0.5)
    ax1.set_title(f"Time Domain: {f_signal}Hz Signal vs {fs}Hz Sampling")
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)

    # --- Frequency Domain ---
    ax2.plot(xf, mag, color='blue', lw=1.5)
    ax2.fill_between(xf, mag, color='blue', alpha=0.1)
    
    # Colored Nyquist Limits
    ax2.axvline(-fn, color='orange', linestyle='--', lw=2.5, label=f'Lower Nyquist ($-f_s/2$)')
    ax2.axvline(fn, color='green', linestyle='--', lw=2.5, label=f'Upper Nyquist ($+f_s/2$)')
    
    # Shade the "Safe" Zone 1
    ax2.axvspan(-fn, fn, color='gray', alpha=0.05)
    
    ax2.set_title("Spectrum: Observe the peak 'bouncing' off the colored limits")
    ax2.set_xlabel("Frequency [Hz]")
    ax2.set_ylabel("Magnitude")
    ax2.set_xlim(-fs, fs)
    ax2.legend(loc='upper right')
    ax2.grid(True, alpha=0.2)

    plt.show()

# UI Setup
f_slider = widgets.IntSlider(min=100, max=4000, step=50, value=600, description='Signal Hz')
fs_slider = widgets.IntSlider(min=500, max=5000, step=100, value=2000, description='Sample Rate')

display(widgets.VBox([f_slider, fs_slider]), 
        widgets.interactive_output(plot_aliasing_colored, {'f_signal': f_slider, 'fs': fs_slider}))

Output()

## 3. FFT Noise Floor and Process Gain

In [30]:
def plot_adc_final(bits, m_power):

    # --- 0. Memory Management ---
    plt.close('all')  # <--- This kills the old figure and resets memory
    # --- 1. Parameters ---
    fs = 4096
    M = 2**m_power 
    t = np.arange(M) / fs
    f_main = 997  # Prime frequency to avoid bin centering artifacts
    signal_amp = 2.5
    adc_range = 2.5 # Full scale peak
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_main * t)
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    # Quantization process
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    
    # --- 3. Normalized FFT Processing ---
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    # Applying a window (Hann) is standard in Master's level to see the floor better
    window = np.hanning(n)
    fft_mag = np.abs(np.fft.rfft(quantized * window)) / (np.sum(window)/2)
    
    # Normalize peak to 0 dBc
    fft_db_raw = 20 * np.log10(fft_mag + 1e-15)
    max_val = np.max(fft_db_raw)
    fft_db_norm = fft_db_raw - max_val
    
    # --- 4. Formula Calculations ---
    sqnr_theory = 6.02 * bits + 1.76
    # Process Gain: FFT spreads noise across N/2 bins
    proc_gain = 10 * np.log10(n / 2)
    # The 'Grass' floor is the SQNR + the Processing Gain
    fft_floor_level = -sqnr_theory - proc_gain
    
    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 7))
    
    plt.plot(freqs, fft_db_norm, color='purple', lw=0.5, label="Quantized Spectrum", alpha=0.7)
    
    # Reference Lines
    plt.axhline(-sqnr_theory, color='red', linestyle='-', lw=2, 
                label=f"Theoretical SQNR ({sqnr_theory:.1f} dB)")
    plt.axhline(fft_floor_level, color='orange', linestyle=':', lw=2.5, 
                label=f"FFT Noise Floor (Proc. Gain: -{proc_gain:.1f} dB)")
    plt.axhline(0, color='black', lw=1.5, label="Signal Peak (0 dBc)")
    
    # Inclusion of SQNR in Title
    plt.title(f"ADC Analysis: SQNR = 6.02({bits}) + 1.76 = {sqnr_theory:.2f} dB\n"
              f"Resolution: {fs/n:.2f} Hz/bin | FFT Process Gain: {proc_gain:.1f} dB", 
              fontsize=13, fontweight='bold')
    
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBc)")
    plt.ylim(-160, 10)
    plt.xlim(0, fs/2)
    plt.grid(True, which='both', linestyle=':', alpha=0.5)
    plt.legend(loc='lower left', framealpha=0.9)
    plt.tight_layout()
    plt.show()

# --- UI Controls ---
bits_s = widgets.IntSlider(min=8, max=14, step=1, value=12, description='ADC Bits')
m_slider = widgets.IntSlider(min=10, max=14, step=1, value=12, description='FFT Size (2^n)')

ui = VBox([HBox([bits_s, m_slider])])
out = interactive_output(plot_adc_final, {'bits': bits_s, 'm_power': m_slider})
display(ui, out)

Output()

## 4. SFDR values

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, interactive_output
from IPython.display import display, clear_output

def plot_adc_master_class(bits, signal_amp, adc_range, m_power):
    # --- 0. Memory Management ---
    plt.close('all') 

    # --- 1. Parameters ---
    fs = 4096
    M = 2**m_power 
    t = np.arange(M) / fs
    f_main = 997.0 
    
    # --- 2. Signal & Quantization ---
    signal = signal_amp * np.sin(2 * np.pi * f_main * t)
    
    v_max, v_min = adc_range, -adc_range
    num_levels = 2**bits
    lsb = (v_max - v_min) / num_levels
    
    # Quantization with Clipping
    indices = np.floor((signal - v_min) / lsb)
    indices = np.clip(indices, 0, num_levels - 1)
    quantized = v_min + (indices + 0.5) * lsb
    
    # --- 3. FFT & Normalization (dBFS) ---
    n = len(quantized)
    freqs = np.fft.rfftfreq(n, 1/fs)
    fft_mag = np.abs(np.fft.rfft(quantized)) / (n/2)
    fft_db_fs = 20 * np.log10(fft_mag / adc_range + 1e-15)
    
    # --- 4. Metrics ---
    carrier_idx = np.argmax(fft_db_fs)
    carrier_level = fft_db_fs[carrier_idx]
    
    # Mask carrier to find highest spur
    fft_no_carrier = fft_db_fs.copy()
    mask_width = 12
    fft_no_carrier[max(0, carrier_idx-mask_width):min(n//2, carrier_idx+mask_width)] = -200
    spur_idx = np.argmax(fft_no_carrier)
    spur_level = fft_db_fs[spur_idx]
    
    # Calculated Values for Title
    sqnr_theo_nyquist = 6.02 * bits + 1.76
    proc_gain = 10 * np.log10(M /2)
    total_expected_sqnr = sqnr_theo_nyquist + proc_gain
    fft_floor = -sqnr_theo_nyquist - proc_gain

    # --- 5. Plotting ---
    clear_output(wait=True)
    plt.figure(figsize=(12, 8))
    
    plt.plot(freqs, fft_db_fs, color='purple', lw=0.7, alpha=0.5, label="Quantized Spectrum")
    
    # Reference Lines
    plt.axhline(0, color='black', lw=2, label="0 dBFS (Full Scale)")
    plt.axhline(spur_level, color='red', linestyle='--', lw=1.5, label=f"SFDR Level ({spur_level:.1f} dBFS)")
    plt.axhline(fft_floor, color='orange', linestyle=':', lw=2, label="FFT 'Grass' Floor")

    # SFDR (dBFS) Annotation - Distance from 0 to Spur (Red)
    sfdr_dbfs = -spur_level
    plt.annotate('', xy=(fs/10, spur_level), xytext=(fs/10, 0),
                  arrowprops=dict(arrowstyle='<->', color='red', lw=1.5))
    plt.text(fs/10 + 50, spur_level/2, f"SFDR: {sfdr_dbfs:.1f} dBFS", 
              color='red', fontweight='bold', verticalalignment='center')

    # SFDR (dBc) Annotation - Distance from Carrier to Spur (Blue)
    sfdr_dbc = carrier_level - spur_level
    plt.annotate('', xy=(freqs[carrier_idx], spur_level), xytext=(freqs[carrier_idx], carrier_level),
                  arrowprops=dict(arrowstyle='<->', color='blue', lw=2))
    plt.text(freqs[carrier_idx] + 50, (carrier_level + spur_level)/2, 
              f"SFDR: {sfdr_dbc:.1f} dBc", color='blue', fontweight='bold')

    # Formatting and Title
    plt.title(f"ADC Performance: {bits}-Bit | FFT Points: {n}\n"
              f"SQNR (Nyquist): {sqnr_theo_nyquist:.1f} dB | Proc. Gain: {proc_gain:.1f} dB | "
              f"Total Expected SQNR: {total_expected_sqnr:.1f} dB", 
              fontsize=13, fontweight='bold')
    
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude (dBFS)")
    plt.ylim(min(fft_floor-20, -160), 15)
    plt.grid(True, alpha=0.2)
    plt.legend(loc='lower left', framealpha=0.9)
    plt.show()

# --- UI Controls ---
b_s = widgets.IntSlider(min=4, max=18, step=1, value=12, description='ADC Bits')
a_s = widgets.FloatSlider(min=0.1, max=10.0, step=0.1, value=4.5, description='Sig Amp (V)')
r_s = widgets.FloatSlider(min=1.0, max=10.0, step=0.5, value=5.0, description='ADC FS (V)')
m_s = widgets.IntSlider(min=12, max=16, step=1, value=12, description='FFT (2^n)')

ui = VBox([HBox([b_s, a_s]), HBox([r_s, m_s])])
out = interactive_output(plot_adc_master_class, {'bits': b_s, 'signal_amp': a_s, 'adc_range': r_s, 'm_power': m_s})

display(ui, out)

Output()